# IMPORT STUFF

In [1]:
from json import dumps, loads
import logging
import mysql.connector
from mysql.connector import errorcode
import os
import pandas as pd
from pathlib import Path
from typing import List, LiteralString, Sequence, Tuple, Union
import time
import traceback

debug_mode = True
if debug_mode:
    import inspect

regenerate_db = False

# CONNECT TO DATABASE

In [2]:
def _connect(host, user, pwd, db=""):
    if db:
        return mysql.connector.connect(host=host, user=user, password=pwd, database=db) 
    return mysql.connector.connect(host=host, user=user, password=pwd)

# Credentials
host, user, pwd = "localhost", "root", "ROOTroot1"

# Database
db_name = "z_scans"

try:
    # Make connection
    connection = _connect(host, user, pwd)
    print(f"Connection established on '{host}' with '{user}' access.")
    connection.autocommit = True
    cursor = connection.cursor()

    # Attempt database creation
    if regenerate_db:
        cursor.execute(f"DROP DATABASE IF EXISTS `{db_name}`;")  # This is only to start a-fresh.
    cursor.execute(f"CREATE DATABASE IF NOT EXISTS `{db_name}`;")  # backticks are for safety if db_name becomes a user input in future
    print(f"Database {db_name} created successfully.")

except mysql.connector.Error as err:
    if err.errno == errorcode.CR_UNKNOWN_HOST:
        print(f"Failed to connect to {host}. Unknown host.")
        exit()
    elif err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Access denied. User name or password is wrong.")
        exit()
    elif err.errno == errorcode.ER_DB_CREATE_EXISTS:
        pass
    else:
        print(err)
        exit()

finally:
    cursor.execute(f"USE `{db_name}`;")
    print(f"Using {db_name} as default database.")

Connection established on 'localhost' with 'root' access.
Database z_scans created successfully.
Using z_scans as default database.


# Show exisiting samples

In [3]:
cursor.execute("SELECT * FROM z_scans.AllSamples;")
results = cursor.fetchall()
for result in results:
    print(result)

(1, 'silica', 'silica', None, None, 3.0, None, None)
(2, 'silica', 'silica', None, None, 4.0, None, None)
(3, 'Butanol', 'solvent', None, None, None, 0.8098, 1.3993)
(4, 'Chloroform', 'solvent', None, None, None, 1.484, 1.4459)
(5, 'DCM', 'solvent', None, None, None, 1.3255, 1.4244)
(6, 'DMF', 'solvent', None, None, None, 0.9445, 1.4305)
(7, 'DMSO', 'solvent', None, None, None, 1.101, 1.4783)
(8, 'DMSO (Deuterated)', 'solvent', None, None, None, 1.19, 1.476)
(9, 'Ethanol', 'solvent', None, None, None, 0.7893, 1.3611)
(10, 'THF', 'solvent', None, None, None, 0.8833, 1.405)
(11, 'Toluene', 'solvent', None, None, None, 0.8623, 1.4967)
(12, 'Water', 'solvent', None, None, None, 1.0, 1.333)
(13, 'Water (Deuterated)', 'solvent', None, None, None, 1.1044, 1.3283)


# simulate contents from the program

In [4]:
float_tolerance = 1E-6

codeOfSamples = {"Silica": "silica",
                "DCM": "solvent", "Water": "solvent",
                "s1": "sample", "s2": "sample", "s3": "sample"}

solvents = {"DCM": {"density": 1.3255, "index": 1.4244},
            "Water": {"density": 1.0, "index": 1.333}}

silica_thicknesses = [3, 4]
silicaThickness_dataSavingTab_doubleSpinBox_value = 3
solventName_dataSavingTab_comboBox_currentText = 'DCM'
concentration_dataSavingTab_doubleSpinBox_value = 1

user_inputs = {'silica': [('silica_thicknessMM', silicaThickness_dataSavingTab_doubleSpinBox_value)],
               'solvent': [('solvent_density', solvents[solventName_dataSavingTab_comboBox_currentText]["density"]),
                            ('solvent_index', solvents[solventName_dataSavingTab_comboBox_currentText]['index'])],
               'sample': [('sample_solvent', solventName_dataSavingTab_comboBox_currentText),
                           ('sample_concentration', concentration_dataSavingTab_doubleSpinBox_value)]
               }

def find_item(codeOfSample, cuvetteType):
    conditions = []
    for param in user_inputs[cuvetteType]:
        if param[1] is not None:
            if isinstance(param[1], (float|int)):
                conditions.append(f"abs({param[0]}-%s) <= {float_tolerance}")
            else:
                conditions.append(f"{param[0]} = %s")
    query_sample_id = f"sample_code = %s AND " + " AND ".join(conditions)
    query_params = (codeOfSample,) + tuple(param[1] for param in user_inputs[cuvetteType] if param[1] is not None)
    
    query = "SELECT * FROM AllSamples WHERE " + query_sample_id + ";"
    
    print(query % query_params)
    
    cursor.execute(query, query_params)
        
    return cursor.fetchone()
    
# this happens on save

for code in codeOfSamples:
    cuvetteType = codeOfSamples[code].lower()
    
    # below should work with the program (only change the nomenclature)
    
    is_found = find_item(code, cuvetteType)  # in terms of the program nomenclature it is:
                                             # (codeOfSample_comboBox.currentText(), cuvetteType_comboBox.currentText())
                                             # and then it should work without the `for` loop
    if not is_found:  # requires adding the sample to database
        print(f"{code=},{cuvetteType=}")
        print("Nothing found")
        print("Getting the parameters from the user inputs.")
        print("Adding to database")
        print("-"*50)
    else:
        print("ID:", is_found[0])
        # should check additional parameters (those that define affiliation of codeOfSample to cuvetteType)
        print(f"{code=},{cuvetteType=}, Checking additional parameters")
        print("-"*50)

SELECT * FROM AllSamples WHERE sample_code = Silica AND abs(silica_thicknessMM-3) <= 1e-06;
ID: 1
code='Silica',cuvetteType='silica', Checking additional parameters
--------------------------------------------------
SELECT * FROM AllSamples WHERE sample_code = DCM AND abs(solvent_density-1.3255) <= 1e-06 AND abs(solvent_index-1.4244) <= 1e-06;
ID: 5
code='DCM',cuvetteType='solvent', Checking additional parameters
--------------------------------------------------
SELECT * FROM AllSamples WHERE sample_code = Water AND abs(solvent_density-1.3255) <= 1e-06 AND abs(solvent_index-1.4244) <= 1e-06;
code='Water',cuvetteType='solvent'
Nothing found
Getting the parameters from the user inputs.
Adding to database
--------------------------------------------------
SELECT * FROM AllSamples WHERE sample_code = s1 AND sample_solvent = DCM AND abs(sample_concentration-1) <= 1e-06;
code='s1',cuvetteType='sample'
Nothing found
Getting the parameters from the user inputs.
Adding to database
------------